
Implementasi jaringan saraf dengan lapisan softmax yang Anda lihat dalam video sebelumnya memang berfungsi dengan baik. Namun, ada cara yang lebih baik untuk mengimplementasikannya. Mari kita lihat apa yang bisa salah dari implementasi tersebut dan bagaimana cara memperbaikinya.

### Masalah dengan Presisi Numerik
Sebagai contoh, kita dapat menghitung nilai $x$ dengan dua cara berbeda:
1. **Cara 1**: Mengatur $x = \frac{2}{10.000}$.
2. **Cara 2**: Mengatur $x = (1 + \frac{1}{10.000}) - (1 - \frac{1}{10.000})$, yang setelah disederhanakan akan memberikan hasil yang sama, yaitu $x = \frac{2}{10.000}$.

Walaupun kedua ekspresi ini secara matematis sama, ketika dihitung dalam komputer, hasil dari cara 2 mungkin akan sedikit berbeda karena adanya **error pembulatan (round-off error)** yang diakibatkan oleh keterbatasan presisi bilangan floating-point di komputer.

<img src="img\image-11.png" alt="Contoh Deteksi Mobil" style="display: block; margin-left: auto; margin-right: auto; width: 800px;">

### Relevansi dengan Softmax
Dalam implementasi lapisan softmax, meskipun secara konsep perhitungan fungsi biaya (cost function) benar, ada cara untuk meminimalkan error pembulatan ini agar perhitungannya lebih akurat. Misalnya, jika nilai $z$ dalam softmax sangat besar atau sangat kecil, maka $e^z$ bisa menjadi terlalu besar atau terlalu kecil, yang dapat menyebabkan ketidakstabilan numerik.

Dengan menggunakan pendekatan yang lebih numerik stabil, kita dapat menghindari perhitungan langsung dari nilai tengah (intermediate value), seperti $a$, yang dalam contoh sebelumnya adalah $1 + \frac{1}{10.000}$ dan $1 - \frac{1}{10.000}$. Sebaliknya, kita langsung mendefinisikan fungsi loss-nya.

### Implementasi Stabil untuk Logistic Regression
Dalam **logistic regression**, perhitungan biasanya dilakukan sebagai berikut:
1. Hitung aktivasi $a$ dengan rumus:
   $$
   a = \frac{1}{1 + e^{-z}}
   $$
2. Hitung loss dengan:
   $$
   \text{loss} = -y \log(a) - (1 - y) \log(1 - a)
   $$

Namun, ada cara lain yang lebih akurat untuk menghitung loss ini. Alih-alih menghitung $a$ terlebih dahulu, kita dapat memasukkan ekspresi untuk $a$ langsung ke dalam perhitungan loss, sehingga TensorFlow dapat mengatur ulang perhitungannya agar lebih akurat.

<img src="img\image-11.png" alt="Contoh Deteksi Mobil" style="display: block; margin-left: auto; margin-right: auto; width: 800px;">

### Peningkatan Implementasi Softmax
Konsep ini juga berlaku untuk **softmax regression**. Dalam perhitungan biasa, kita menghitung aktivasi untuk setiap kelas $a_1, a_2, \dots, a_n$ dengan rumus:
$
a_j = \frac{e^{z_j}}{\sum_{k=1}^{n} e^{z_k}}
$
Kemudian kita hitung loss berdasarkan nilai $y$ yang benar:
$
\text{loss} = -\log(a_y)
$

Namun, alih-alih menghitung $a_j$ terlebih dahulu, kita bisa langsung mendefinisikan loss-nya:
$
\text{loss} = -\log\left(\frac{e^{z_y}}{\sum_{k=1}^{n} e^{z_k}}\right)
$
Pendekatan ini memberi TensorFlow fleksibilitas untuk menghitung loss dengan lebih akurat tanpa perlu menghitung $a_j$ secara eksplisit.

### Contoh Kode dengan TensorFlow
Di TensorFlow, kita dapat menggunakan argumen `from_logits=True` untuk memberitahu TensorFlow bahwa kita ingin menghitung loss langsung dari nilai $z$, bukan dari probabilitas yang telah di-softmax-kan. Ini membantu mengurangi **error pembulatan**.

Contoh kodenya adalah sebagai berikut:

```python
model.compile(
    optimizer='adam', 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
)
```

Dengan menggunakan from_logits=True, TensorFlow akan menghitung softmax secara internal dengan cara yang lebih stabil.

Kesimpulan
Walaupun implementasi awal softmax bekerja dengan baik, pendekatan yang lebih numerik stabil seperti ini disarankan untuk menghindari error pembulatan yang bisa menjadi masalah, terutama pada kasus-kasus dengan angka sangat besar atau sangat kecil. Ini membuat model menjadi lebih akurat dan efisien.

Dengan menggunakan teknik ini, Anda dapat mengurangi **error pembulatan** dalam perhitungan fungsi biaya untuk jaringan saraf yang menggunakan softmax regression, terutama ketika bekerja dengan TensorFlow.
